## Merlin Setup

The goal of this notebook is to create a user-managed container that can be used in Vertex Workbench

### Setup variables, paths, and create artifact registry

In [1]:
VERSION=22.12
REPO_NAME="workbench"
REGION="us-central1"
PROJECT="hybrid-vertex" # TODO: update with your project_id
IMAGE_ID="tensorflow"
MERLIN_IMAGE_NAME=f"{REGION}-docker.pkg.dev/{PROJECT}/{REPO_NAME}/merlin-{IMAGE_ID}-{VERSION}"

MERLIN_CONTAINER="merlin_container"

!mkdir {MERLIN_CONTAINER}

!gcloud beta artifacts repositories create {REPO_NAME} \
    --repository-format=docker \
    --location=$REGION

mkdir: cannot create directory ‘merlin_container’: File exists
ERROR: (gcloud.beta.artifacts.repositories.create) ALREADY_EXISTS: the repository already exists


In [2]:
!gcloud config set project $PROJECT

Updated property [core/project].


#### Find your service account for you tenant project - usually project_id-compute@developer.gserviceaccount.com

In [3]:
# !gcloud projects add-iam-policy-binding hybrid-vertex --member=serviceAccount:xxxxxxxx-compute@developer.gserviceaccount.com --role=roles/artifactregistry.admin

In [4]:
!gcloud auth configure-docker us-central1-docker.pkg.dev --quiet


{
  "credHelpers": {
    "gcr.io": "gcloud",
    "us.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "asia.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud"
  }
}
Adding credentials for: us-central1-docker.pkg.dev
Docker configuration file updated.


### Create Docker image derived from Deeplearning containers

In [5]:
%%writefile {MERLIN_CONTAINER}/jupyter_notebook_config.py
c.NotebookApp.ip = '*'
c.NotebookApp.token = ''
c.NotebookApp.password = ''
c.NotebookApp.open_browser = False
c.NotebookApp.port = 8080
c.NotebookApp.terminado_settings = {'shell_command': ['/bin/bash']}
c.NotebookApp.allow_origin_pat = (
'(^https://8080-dot-[0-9]+-dot-devshell\.appspot\.com$)|'
'(^https://colab\.research\.google\.com$)|'
'((https?://)?[0-9a-z]+-dot-(?:us|asia|europe|northamerica|southamerica)-?[0-9a-z]+\.notebooks\.googleusercontent.com)')
c.NotebookApp.allow_remote_access = True
c.NotebookApp.disable_check_xsrf = False

Overwriting merlin_container/jupyter_notebook_config.py


In [10]:
%%writefile {MERLIN_CONTAINER}/Dockerfile
FROM nvcr.io/nvidia/merlin/merlin-tensorflow:22.12
RUN echo "deb [signed-by=/usr/share/keyrings/cloud.google.gpg] http://packages.cloud.google.com/apt cloud-sdk main" | tee -a /etc/apt/sources.list.d/google-cloud-sdk.list && curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key --keyring /usr/share/keyrings/cloud.google.gpg  add - && apt-get update -y && apt-get install google-cloud-sdk -y
EXPOSE 8080

# copy the dependencies file to the working directory
COPY merlin_container/requirements.txt .
# install dependencies
RUN apt-get install nodejs npm -y
RUN pip install -r requirements.txt
#RUN mkdir /root/.jupyter
             
COPY merlin_container/jupyter_notebook_config.py /root/.jupyter

ENV pwd=""
ENTRYPOINT exec jupyter-lab --ip=0.0.0.0 --port=8080 --no-browser --allow-root --ServerApp.allow_origin="*" --NotebookApp.token="$pwd" --NotebookApp.password="$pwd"

Overwriting merlin_container/Dockerfile


In [11]:
%%writefile {MERLIN_CONTAINER}/requirements.txt
fastapi
merlin-models
gsutil
gcsfs
matplotlib
google-cloud-aiplatform
jupyterlab-git

Overwriting merlin_container/requirements.txt


In [ ]:
!docker build . -f $MERLIN_CONTAINER/Dockerfile -t $MERLIN_IMAGE_NAME
!docker push $MERLIN_IMAGE_NAME

Sending build context to Docker daemon  2.795MB
Step 1/9 : FROM nvcr.io/nvidia/merlin/merlin-tensorflow:22.12
22.12: Pulling from nvidia/merlin/merlin-tensorflow

d16dc43b: Pulling fs layer 
6ecee9ab: Pulling fs layer 
e0981273: Pulling fs layer 
03fa67ca: Pulling fs layer 
7f8e4744: Pulling fs layer 
73502f86: Pulling fs layer 
cf4cbe13: Pulling fs layer 
d495b912: Pulling fs layer 
6cd12433: Pulling fs layer 
3d945bde: Pulling fs layer 
5d54675a: Pulling fs layer 
25844090: Pulling fs layer 
a25c21ff: Pulling fs layer 
b700ef54: Pulling fs layer 
21e94f24: Pulling fs layer 
1b6fd612: Pulling fs layer 
1b6fd612: Waiting fs layer 
1cba559e: Pulling fs layer 
054fc370: Pulling fs layer 
c99b1680: Pulling fs layer 
73498bf6: Pulling fs layer 
fd996372: Pulling fs layer 
c9ed50ff: Pulling fs layer 
db3dcf22: Pulling fs layer 
a7f6f15d: Pulling fs layer 
ed83fb5b: Pulling fs layer 
f4c1fb4d: Pulling fs layer 
fde98e21: Pulling fs layer 
e75d70d4: Pulling fs layer 
c32a822c: Pulling fs laye